# IMPORT LIBRARY

In [1]:
import os
import sys
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

2024-02-22 10:24:17.619362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 10:24:17.710734: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-22 10:24:17.714544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64:
2024-02-22 10:24:17.714553: I tensorflow/stream_executor

# DECLARE VARIABLE

In [2]:
PIPELINE_NAME = "heart-failure-pipeline"

# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TRAINER_MODULE_FILE = "modules/trainer.py"
TUNER_MODULE_FILE = "modules/tuner.py"
# requirement_file = os.path.join(root, "requirements.txt")

# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

# Initiation Pipeline

In [3]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

# Run Code

In [4]:
if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components

    components = init_components(
        DATA_ROOT,
        transform_module=TRANSFORM_MODULE_FILE,
        tuner_module=TUNER_MODULE_FILE,
        training_module=TRAINER_MODULE_FILE,
        serving_model_dir=serving_model_dir,
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'num_hidden_layers', 'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}}], 'values': {'num_hidden_layers': 2, 'dense_units': 128, 'learning_rate': 0.001, 'dropout_rate': 0.1}}
INFO:absl:Best Hyperparameters are written to output/heart-failure-pipeline/Tuner/best_hyperparameters/7/best_hyperparameters.txt.
INFO:absl:Tuner results are written to output/heart-failure-pipel

Trial 100 Complete [00h 00m 01s]
val_binary_accuracy: 0.8326562643051147

Best val_binary_accuracy So Far: 0.8665624856948853
Total elapsed time: 00h 01m 22s
Results summary
Results in output/heart-failure-pipeline/Tuner/.system/executor_execution/7/.temp/7/kt_randomsearch
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 006 summary
Hyperparameters:
num_hidden_layers: 2
dense_units: 128
learning_rate: 0.001
dropout_rate: 0.1
Score: 0.8665624856948853

Trial 026 summary
Hyperparameters:
num_hidden_layers: 1
dense_units: 64
learning_rate: 0.01
dropout_rate: 0.2
Score: 0.8575000166893005

Trial 029 summary
Hyperparameters:
num_hidden_layers: 3
dense_units: 160
learning_rate: 0.0001
dropout_rate: 0.1
Score: 0.8571875095367432

Trial 055 summary
Hyperparameters:
num_hidden_layers: 3
dense_units: 224
learning_rate: 0.001
dropout_rate: 0.5
Score: 0.8570312261581421

Trial 061 summary
Hyperparameters:
num_hidden_layers: 3
dense_units: 96
learning_rate: 0.000

INFO:absl:node Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "heart-failure-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20240222-102419.806331"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "heart-failure-pipeline.Trainer"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channels {
        producer_node_query {
          id: "Transform"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "heart-failure-pipeline"
            }
          }
        

Processing ./output/heart-failure-pipeline/_wheels/tfx_user_code_Trainer-0.0+65fbe24679d142252512b2090985c92f5390e81a2d43c33912919881683c28a6-py3-none-any.whl


INFO:absl:Successfully installed 'output/heart-failure-pipeline/_wheels/tfx_user_code_Trainer-0.0+65fbe24679d142252512b2090985c92f5390e81a2d43c33912919881683c28a6-py3-none-any.whl'.
INFO:absl:Training model.


...

INFO:absl:Training complete. Model written to output/heart-failure-pipeline/Trainer/model/8/Format-Serving. ModelRun written to output/heart-failure-pipeline/Trainer/model_run/8
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 8 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "output/heart-failure-pipeline/Trainer/model_run/8"
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "output/heart-failure-pipeline/Trainer/model/8"
, artifact_type: name: "Model"
base_type: MODEL
)]}) for execution 8
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Trainer is finished.
INFO:absl:node Evaluator is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.evaluator.component.Evaluator"
    base_type: EVALUATE
  }
  id: "Evaluator"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 0.0001,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "HeartDisease"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'fairness_indicator_thresholds': 'null', 'example_splits': 'null'} 'custom_extractors'
INFO:absl:Request was made

INFO:absl:Evaluation complete. Results written to output/heart-failure-pipeline/Evaluator/evaluation/9.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to output/heart-failure-pipeline/Evaluator/blessing/9.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 9 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'blessing': [Artifact(artifact: uri: "output/heart-failure-pipeline/Evaluator/blessing/9"
, artifact_type: name: "ModelBlessing"
)], 'evaluation': [Artifact(artifact: uri: "output/heart-failure-pipeline/Evaluator/evaluation/9"
, artifact_type: name: "ModelEvaluation"
)]}) for execution 9
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Evaluator is finished.
INFO:absl:node Pusher is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.pusher.component.Pusher"
    base_type: DEPLOY
  }
  id: "Pusher"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
